In [1]:
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Activation, Dot, Flatten, Reshape, MaxPooling1D, Dropout
from keras.optimizers import Adam
from keras.backend import l2_normalize
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras.callbacks import EarlyStopping
import typing

import random
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from datagenerator import DataGenerator
from kerastuner.tuners import RandomSearch

/var/folders/y3/w8y39yms09jg30l75ycdw56h0000gn/T/ipykernel_7697/2405208978.py:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len (physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

data_folder = os.getcwd() + '/data/eeg'
all_files = [f for f in os.listdir(data_folder) if f.endswith('.npz')]
train_generator = DataGenerator(all_files, batch_size=10, stage='train')
val_generator = DataGenerator(all_files, batch_size=10, stage='val')

time_window = train_generator.time_window
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

def build_model(hp):
    eeg = Input(shape=(time_window, 64))
    env1 = Input(shape=(time_window, 1))
    env2 = Input(shape=(time_window, 1))

    conv1 = Conv1D(filters=hp.Int('filters', min_value=8, max_value=32, step=4),
                   kernel_size=hp.Int('kernel_size', min_value=8, max_value=32, step=4),
                   activation='relu',
                   padding='same')(eeg)

    cos_sim1 = tf.keras.layers.Dot(axes=(1, 1), normalize=True)([conv1, env1])
    cos_sim2 = tf.keras.layers.Dot(axes=(1, 1), normalize=True)([conv1, env2])

    out1 = Dense(1, activation="sigmoid")(tf.keras.layers.Flatten()(tf.keras.layers.Concatenate()([cos_sim1, cos_sim2])))

    out = tf.keras.layers.Reshape([1], name="output_name")(out1)
    model = tf.keras.Model(inputs=[eeg, env1, env2], outputs=[out])

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


Num GPUs Available:  1


In [3]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='output',
    project_name='my_project')

tuner.search_space_summary()

# Start the search
tuner.search(train_generator,
             epochs=50,
             validation_data=val_generator,
             callbacks=[early_stopping])

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 5 Complete [01h 08m 34s]
val_accuracy: 0.7099704146385193

Best val_accuracy So Far: 0.7099704146385193
Total elapsed time: 05h 54m 11s
INFO:tensorflow:Oracle triggered exit


In [5]:
print(best_hyperparameters.values)
print(best_model.summary())

{'filters': 28, 'kernel_size': 32, 'learning_rate': 0.01}
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 64)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 640, 28)      57372       ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 640, 1)]     0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 640, 1)]     0           []                               
                                    

: 